In [ ]:
import os
import cv2
import json
import random
import logging
import numpy as np
import pandas as pd
from tensorflow.keras.utils import load_img
import tensorflow as tf

from IPython.display import clear_output

clear_output()

In [ ]:
video_number = '003'

In [ ]:
def compute_error(y, y_pred):
    """
    Since the competion computes the error based on Euclidian distances, this function computes
    the error for predicted and ground truth results.
    
    @param: list y. a (1, 2) list with original coordinations.
    @param: list y_pred. a (1, 2) list containing the predicted coordinations.
    
    @return: float error. The Euclidian distance between these two given datapoints.
    """
    error = np.sqrt(np.abs(y[0] - y_pred[0]) ** 2 + np.abs(y[1] - y_pred[1]) ** 2)
    return error

In [ ]:
def compute_original_coordination(predicted_coordination, original_box_coordination):
    """
    Computes the coordinations of the predicted Hexbug's head's coordination in the original image
    using the bounding box coordinations.
    
    @param: list predicted_coordination. a (1, 2) list with (x, y) format.
    @param: list original_box_coordination. a (1, 2) list with (x, y) format of bounding box.
    
    @return: list. Coordinations of the predicted Hexbug's head's coordination in the original image
    """
    return (predicted_coordination[0] + original_box_coordination[0], predicted_coordination[1] + original_box_coordination[1])

In [ ]:
reconstructed_model = tf.keras.models.load_model("resnet50_trained_model_data/v1.2/checkpoint_callback/")
clear_output()

In [ ]:
data = pd.read_csv(f'Test_data/test{video_number}_prep.csv').drop(['Unnamed: 0'], axis=1)
data.head()

In [ ]:
result_array = []

# os.mkdir(f'Test_data/annotated_images/{video_number}')

for idx, frame_name in enumerate(os.listdir(f'Test_data/cropped_bugs/{video_number}')):
    img_path = f'Test_data/cropped_bugs/{video_number}/{frame_name}'
    
    original_image_path = f'Test_data/Frames/{video_number}/{frame_name}'
    
    sample_id = int(frame_name.split('.')[0][5:])
    
    img = load_img(img_path)
    np_img = np.array(img)

    expanded_img = np.expand_dims(np_img, 0)
    predicted = list(map(int, reconstructed_model.predict(expanded_img)[0]))
    
    radius = 10
    color = (0, 255, 0)
    thickness = -1
    
    cv2.circle(np_img, predicted, radius, color, thickness)
    cv2.imshow('sample', np_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    # Computes the coordination of the Hexbug's head in the original image.
#     scaled_coordinations = compute_original_coordination(predicted, (data.iloc[sample_id].OriginalBoxCoordinationX1, data.iloc[sample_id].OriginalBoxCoordinationY1))
    
#     result_array.append([sample_id, 0, scaled_coordinations[0], scaled_coordinations[1]])
    
    # Reads and converts the original image into numpy array.
#     original_sample_img = np.array(load_img(original_image_path))

#     cv2.circle(original_sample_img, scaled_coordinations, radius, color, thickness)
    
#    print('------------------------------------------------------------------------')
    
#     if not os.path.exists('annotated_images'):
#         os.mkdir('annotated_images')
    
#     cv2.imshow(f'Sample ID. {frame_name}', original_sample_img)
    
#     cv2.imwrite(f'Test_data/annotated_images/{video_number}/{frame_name}', original_sample_img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

# clear_output()

In [ ]:
import exportTool as export

In [ ]:
tmp = [] # you can skip this step
for val in result_array: 
    tmp = export.fromArrayToDict(val,tmp) #  fromArrayToDict(val)

export.saveList(tmp, f"Test_data/test{video_number}_results.csv")